In [1]:
from pymongo import MongoClient
import json
import re
import csv
from bson import ObjectId
import ast
import string
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Create a MongoClient object and specify the MongoDB connection URL
url = "mongodb://localhost:27017/"
client = MongoClient(url)

# Access a specific database
db = client["YT_db"]

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


# Reading twitter and the updated yt test collection

In [2]:
tw_collection = db.tw_collection

In [3]:
yt_test_collection = db.yt_test_collection


In [7]:
yt_test_collection.find_one()

{'_id': ObjectId('6471c55c3286bb1167a527bf'),
 'id': '7A9D8QfCEKQ',
 'lista_url': "['https://youtu.be/7A9D8QfCEKQ']",
 'tweet_ids': ['1320856307394473984',
  '1321399262551265280',
  '1320914155566731265',
  '1321000433779904517',
  '1321616034999291904',
  '1321400816218050560',
  '1320810069714472966',
  '1320810766711312387']}

# Retrieving tweets text and hashtags associated to each test yt video

In [11]:
def update_video_tweets(yt_collection, tw_collection):
    video_ids = yt_collection.find()  
    i = 0
    
    for video in video_ids:
        if i%10==0:
            print(i)
        i+=1
        video_id = video["id"] 
        
        #tweet_ids_str = video.get("tweet_ids", "")
        #tweet_ids = ast.literal_eval(tweet_ids_str)

        tweet_ids = video.get("tweet_ids", "")

        # Retrieve corresponding tweets from tw_collection
        tweets = tw_collection.find(
            {"tweetid": {"$in": tweet_ids}}, 
            {"text": 1, "hashtag": 1} 
        )

        # Collect text and hashtag for each tweet
        tweet_text = " "
        for tweet in tweets:             
            text = tweet.get("text") 
            if len(text) > 4:
                tweet_text += " " + text

            hashtag = tweet.get("hashtag")
            if len(hashtag) > 4:
                tweet_text += " " + hashtag

        # Update the video document with the tweet_text field
        yt_collection.update_one(
            {"id": video_id},
            {"$set": {"tweet_text": tweet_text}}
        )


In [12]:
update_video_tweets(yt_collection=yt_test_collection, tw_collection=tw_collection)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900


In [14]:
yt_test_collection.find_one()

{'_id': ObjectId('6471c55c3286bb1167a527bf'),
 'id': '7A9D8QfCEKQ',
 'lista_url': "['https://youtu.be/7A9D8QfCEKQ']",
 'tweet_ids': ['1320856307394473984',
  '1321399262551265280',
  '1320914155566731265',
  '1321000433779904517',
  '1321616034999291904',
  '1321400816218050560',
  '1320810069714472966',
  '1320810766711312387'],
 'tweet_text': '  BIDEN FAMILY BUSINESS PARTNER "TONY BOBULINSKI" ENDS BIDEN LEGACY! https://t.co/8OlRTSK7pj via @YouTube @realDonaldTrump   Bring the rain.  https://t.co/g4XPjgssPL @TCscoops13 @cooper_m @realDonaldTrump Shocking stuff- Biden’s are as bad as the Clintons.. https://t.co/uNMBXdfC9G @tominkerry @SamCronin16 @cooper_m Biden’s are as bad as the Clintons, corrupt as fluck https://t.co/uNMBXdfC9G https://t.co/WNvpwHaVO8 Tony B. re Biden BIDEN FAMILY BUSINESS PARTNER "TONY BOBULINSKI" ENDS BIDEN LEGACY! https://t.co/J9JRIcpsEG via @YouTube RT @kunkelKAG: BIDEN FAMILY BUSINESS PARTNER "TONY BOBULINSKI" ENDS BIDEN LEGACY! https://t.co/J9JRIcpsEG via @Yo

# Saving as a json

In [13]:

# Custom JSON encoder class to handle ObjectId serialization
class JSONEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, ObjectId):
            return str(o)
        return super().default(o)

# Retrieve all documents from the collection
all_documents = list(yt_test_collection.find())

# Specify the path and filename for the output JSON file
output_file = "yt_collection_test_text.json"

# Save the collection data to the JSON file
with open(output_file, "w") as file:
    json.dump(all_documents, file, cls=JSONEncoder,indent=4)

print("Collection data saved as JSON.")

Collection data saved as JSON.


# Saving as a csv

In [16]:
# Retrieve all documents from the collection
cursor = yt_test_collection.find()

# Convert the cursor to a DataFrame
df = pd.DataFrame(list(cursor))

# Specify the columns to include in the CSV file
columns = ['id', 'tweet_ids', 'tweet_text']

# Save the DataFrame as a CSV file
df.to_csv('merged_yt_collection_test_text.csv', columns=columns, index=False)

# Preprocess text

In [19]:
def preprocess_text(text):
    # Tokenize the text into individual words
    tokens = word_tokenize(text.lower())

    # Remove the punctuation
    table = str.maketrans('', '', string.punctuation)
    tokens = [token.translate(table) for token in tokens if token.isalpha()]

    # Remove the English stop words
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # Using a word Lemmatizer 
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    preprocessed_text = ' '.join(tokens)

    return preprocessed_text

In [20]:
df['preprocessed_text'] = df['tweet_text'].apply(lambda x: preprocess_text(x))

# Saving as a new csv

In [21]:
df.to_csv("merged_yt_collection_test_preprocessedtxt.csv",index=False)